In [1]:
from xml.etree import ElementTree as ET
import random

In [2]:
def Xml_Preview(file_path):
  # Load and parse the XML file
  tree = ET.parse(file_path)
  root = tree.getroot()

  for child in list(root)[:10]:  # Limiting to first 10 for overview
      print(ET.tostring(child, encoding='unicode'))

  # Checking how many blocks or similar elements are present
  blocks = root.findall('.//GameObjects/Block')
  print(f"Total number of blocks: {len(blocks)}")

In [3]:
Xml_Preview('../main_set/level_0.xml')

<Camera x="0" y="1" minWidth="20" maxWidth="30" />
  
<Birds>
   <Bird type="BirdRed" />
   <Bird type="BirdRed" />
   <Bird type="BirdRed" />
  </Birds>
  
<Slingshot x="-8" y="-2.5" />
  
<GameObjects>
   <Block type="SquareHole" material="ice" x="3.38" y="-2.83" rotation="0.0" />
   <Block type="RectFat" material="ice" x="3.8" y="-1.78" rotation="0.0" />
   <Block type="RectFat" material="ice" x="5.13" y="-2.48" rotation="90.0" />
   <Block type="SquareSmall" material="ice" x="3.52" y="-2.2" rotation="0.0" />
   <Block type="SquareTiny" material="stone" x="5.2" y="-1.92" rotation="0.0" />
   <Block type="SquareTiny" material="ice" x="5.4799999999999995" y="-2.76" rotation="0.0" />
   <Block type="SquareTiny" material="wood" x="5.97" y="-2.9699999999999998" rotation="0.0" />
   <Block type="RectTiny" material="wood" x="2.54" y="-3.1799999999999997" rotation="0.0" />
   <Block type="RectTiny" material="stone" x="4.08" y="-3.1799999999999997" rotation="0.0" />
   <Block type="RectTiny"

In [4]:
def remove_random_non_free_standing_blocks(xml_file, destination, blocks_to_remove=1, type='n'):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all blocks
    blocks = root.findall('.//GameObjects/Block')

    # Create a list of all blocks' positions (x, y)
    positions = {block: (float(block.get('x')), float(block.get('y'))) for block in blocks}

    # Identify free-standing blocks (no blocks directly above them)
    # A block is considered to have another block directly above it if there's any block
    # within a small x-range (to account for blocks that are not exactly aligned)
    # and with a y-value greater than the current block's y-value.
    x_tolerance = 0.5  # Tolerance for determining if blocks are "above" each other
    non_free_standing_blocks = []
    for block, (x, y) in positions.items():
        if any(
            abs(x - other_x) < x_tolerance and other_y > y
            for other_block, (other_x, other_y) in positions.items() if other_block != block
        ):
            non_free_standing_blocks.append(block)

    # If the number of blocks to remove is greater than the number of free-standing blocks, adjust it
    if type == 'r':
      blocks_to_remove = int(len(blocks) * blocks_to_remove)

    blocks_to_remove = min(blocks_to_remove, len(non_free_standing_blocks))

    # Randomly select free-standing blocks to remove
    blocks_to_remove = random.sample(non_free_standing_blocks, blocks_to_remove) if non_free_standing_blocks else []

    # Remove the selected blocks
    for block in blocks_to_remove:
        root.find('.//GameObjects').remove(block)

    # Create a new XML file with the modified structure
    # write with a new name in destination
    
    # new_file_path = xml_file.replace('.xml', '_modified.xml')
    new_file_path = xml_file.replace('.xml', '.xml')
    new_file_path = destination + new_file_path.split('\\')[-1]
    tree.write(new_file_path)

    return new_file_path

In [5]:
import os
for file in os.listdir('../single_structure'):
    if file.endswith('.xml'):
        xml_file = os.path.join('../single_structure', file)
        new_file = remove_random_non_free_standing_blocks(xml_file, '../modified_set/', 1) # block to remove
        # new_file = remove_random_non_free_standing_blocks(xml_file, '../modified_set/', 0.1, 'r')
        # print(f"New file created: {new_file}")